In [2]:
import os
import sys
import csv
import ccxt  # noqa: E402

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, data):
    with open(filename, mode='w') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows([['date', 'open', 'high', 'low', 'close', 'volume']])
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

scrape_candles_to_csv('bitfinex.csv', 'bitfinex', 3, 'BTC/USDT', '1h', '2000-01-01T00:00:00Z', 100)

100 candles in total from 2020-01-20T15:00:00.000Z to 2020-01-24T18:00:00.000Z
200 candles in total from 2020-01-16T11:00:00.000Z to 2020-01-24T18:00:00.000Z
300 candles in total from 2020-01-12T07:00:00.000Z to 2020-01-24T18:00:00.000Z
400 candles in total from 2020-01-08T03:00:00.000Z to 2020-01-24T18:00:00.000Z
500 candles in total from 2020-01-03T23:00:00.000Z to 2020-01-24T18:00:00.000Z
600 candles in total from 2019-12-30T19:00:00.000Z to 2020-01-24T18:00:00.000Z
700 candles in total from 2019-12-26T15:00:00.000Z to 2020-01-24T18:00:00.000Z
800 candles in total from 2019-12-22T11:00:00.000Z to 2020-01-24T18:00:00.000Z
900 candles in total from 2019-12-18T07:00:00.000Z to 2020-01-24T18:00:00.000Z
1000 candles in total from 2019-12-14T03:00:00.000Z to 2020-01-24T18:00:00.000Z
1100 candles in total from 2019-12-09T23:00:00.000Z to 2020-01-24T18:00:00.000Z
1200 candles in total from 2019-12-05T19:00:00.000Z to 2020-01-24T18:00:00.000Z
1300 candles in total from 2019-12-01T15:00:00.00